<a href="https://colab.research.google.com/github/raj166/Machine-Learning-Basic-models/blob/main/Market_Basket/market_basket_analysis_tfifd_cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('transactions_by_dept.csv')
data

,POS Txn,Dept,ID,Sales U
0,16120100160021008773,0261:HOSIERY,250,2
1,16120100160021008773,0634:VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,0879:PET SUPPLIES,158,2
3,16120100160021008773,0973:CANDY,175,2
4,16120100160021008773,0982:SPIRITS,176,1
...,...,...,...,...
4534,16120100160162000843,0857:PC PERIPH/OFFICE ACC,155,1
4535,16120100160162000845,0395:MENS ATHLETIC SHOES,49,1
4536,16120100160162000845,0815:AUDIO ELECTRONICS,132,1
4537,16120100160162000846,0532:AMERICAN GREETINGS,72,1


In [ ]:
data['Dept'] = data['Dept'].apply(lambda x: x[5:])
data

,POS Txn,Dept,ID,Sales U
0,16120100160021008773,HOSIERY,250,2
1,16120100160021008773,VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,PET SUPPLIES,158,2
3,16120100160021008773,CANDY,175,2
4,16120100160021008773,SPIRITS,176,1
...,...,...,...,...
4534,16120100160162000843,PC PERIPH/OFFICE ACC,155,1
4535,16120100160162000845,MENS ATHLETIC SHOES,49,1
4536,16120100160162000845,AUDIO ELECTRONICS,132,1
4537,16120100160162000846,AMERICAN GREETINGS,72,1


In [ ]:
for i in list(data.columns):
  print(i, data[i].nunique())

POS Txn 2064
Dept 160
ID 160
Sales U 28


In [ ]:
grouped_data = pd.DataFrame(data.groupby('POS Txn'))
grouped_data.columns = ["ID", "dept"]
grouped_data

,ID,dept
0,16120100160021008773,POS Txn Dept ...
1,16120100160021008774,POS Txn Dept ID S...
2,16120100160021008775,POS Txn Dept I...
3,16120100160021008776,POS Txn Dept ...
4,16120100160021008777,POS Txn Dept ID Sales ...
...,...,...
2059,16120100160162000842,POS Txn Dept ...
2060,16120100160162000843,POS Txn De...
2061,16120100160162000845,POS Txn Dep...
2062,16120100160162000846,POS Txn Dept...


In [ ]:
print(list(list(grouped_data.iloc[0])[1]['ID']))
print(list(list(grouped_data.iloc[0])[1]['Dept']))

[250, 102, 158, 175, 176, 177, 179]
['HOSIERY', 'VITAMINS & HLTH AIDS', 'PET SUPPLIES', 'CANDY', 'SPIRITS', 'WINE', 'TOBACCO']


In [ ]:
#removing rows with only one item

count = []
for i in range(len(grouped_data)):
  if len(list(grouped_data.iloc[i])[1]) == 1:
    count.append(i)

print(len(count))
grouped_data = grouped_data.drop(labels=count, axis=0)

977


In [ ]:
for i in range(len(grouped_data)):
  grouped_data["dept"].iloc[i] = ' '.join(list(grouped_data.iloc[i][1]['Dept']))

grouped_data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ID,dept
0,16120100160021008773,HOSIERY VITAMINS & HLTH AIDS PET SUPPLIES CAND...
1,16120100160021008774,HEALTH AIDS PERSONAL CARE
2,16120100160021008775,PRE-RECORDED A/V SMALL ELECTRICS SPIRITS
5,16120100160021008778,SPIRITS TOBACCO
6,16120100160021008779,PET SUPPLIES SPIRITS WINE BEER
...,...,...
2053,16120100160162000835,PEGABLES BABY CARE
2055,16120100160162000837,GENERAL HOUSEWARES BATH
2057,16120100160162000840,CAMERAS & EQUIP TELEVISION AUDIO ELECTRONICS P...
2058,16120100160162000841,SLEEPWEAR/LOUNGEWEAR GENERAL HOUSEWARES AS SEE...


In [ ]:
grouped_data.head(5)

,ID,dept
0,16120100160021008773,HOSIERY VITAMINS & HLTH AIDS PET SUPPLIES CAND...
1,16120100160021008774,HEALTH AIDS PERSONAL CARE
2,16120100160021008775,PRE-RECORDED A/V SMALL ELECTRICS SPIRITS
5,16120100160021008778,SPIRITS TOBACCO
6,16120100160021008779,PET SUPPLIES SPIRITS WINE BEER


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
dept_matrix = tfidf.fit_transform(grouped_data["dept"])

cosine_similarities = cosine_similarity(dept_matrix)

In [ ]:
similarities = {}

for i in range(len(cosine_similarities)):
  similar_indices = cosine_similarities[i].argsort()[:-7:-1]
  similarities[grouped_data["ID"].iloc[i]] = [(cosine_similarities[i][x], grouped_data["ID"].iloc[x]) for x in similar_indices][1:]

In [ ]:
# user = grouped_data.iloc[[40]]
user = {
    "ID": 123,
    "dept": ['SHOES'],
}
user

{'ID': 123, 'dept': ['SHOES']}

In [ ]:
new_basket_matrix = tfidf.transform(user['dept'])

cos_similarity_tfidf = map(lambda x: cosine_similarity(new_basket_matrix, x), dept_matrix)
output2 = list(cos_similarity_tfidf)

In [ ]:
top = sorted(range(len(output2)), key=lambda i: output2[i][0], reverse=True)[:30]

list_scores = [output2[i][0] for i in top]

[(list_scores[i][0], grouped_data["dept"].iloc[top[i]]) for i in range(len(list_scores))]

[(0.7441111271653983, 'MENS SHOES BEVERAGES'),
 (0.651610258976043, 'WOMENS ATHLTIC SHOES MENS ATHLETIC SHOES'),
 (0.5952206748426037, 'MENS SHOES BEAUTY CARE BEER'),
 (0.5743886788178663, 'MENS DENIM JEANS MENS SHOES MENS ATHLETIC SHOES'),
 (0.5018953434026269, 'MENS FURNISHINGS MENS SHOES CANDY'),
 (0.48438467840826405, 'MENS ATHLETIC SHOES GENERAL HOUSEWARES'),
 (0.48389024417084237, 'MENS SHOES CHILDRENS FOOTWEAR'),
 (0.4807789951531046, 'MENS ATHLETIC SHOES GENERAL GROCERIES SPIRITS'),
 (0.4783250237540196,
  'MENS SHOES HEALTH AIDS PERSONAL CARE HOUSEHOLD CLEANING BEVERAGES'),
 (0.4769792925888857, 'WOMENS ATHLTIC SHOES MENS SHOES DIMAGE/PHONE/GPS/ACC'),
 (0.46998677906741504, 'MENS ATHLETIC SHOES HEALTH AIDS BEAUTY CARE'),
 (0.45858264807018523,
  'MENS FURNISHINGS MENS SHOES MENS ATHLETIC SHOES BOYS ACCESSORIES BEAUTY CARE TELEVISION AS SEEN ON TV PET SUPPLIES'),
 (0.4278544997184094, 'MENS FURNISHINGS MENS ATHLETIC SHOES CANDY'),
 (0.42452024293232987, 'YOUNG MENS MENS ATHLETI